GENERAL
=============================

**Description**
- Prepara un único dataset definitivo para modelado que combine información relevante de las tablas disponibles.
- Incluye las features que se han calculado previamente.

- Aplica transformaciones adecuadas a las variables categóricas y a las variables numéricas (si lo consideras necesario) 
  para dejar los datos listos para ser utilizados por un modelo de machine learning.

- Justifica las transformaciones realizadas. La variable objetivo es TotalPriceCalculated, por lo que debe quedar sin transformaciones.

- El campo TotalPrice en la tabla sales no tiene valores válidos. Utilizando la información de precios de la tabla products, 
  calcula el valor real de la venta para cada registro y almacena en una nueva columna. Utiliza la siguiente fórmula: TotalPriceCalculated=(Quantity×UnitPrice)×(1−Discount)

In [99]:
import pandas as pd
import numpy as np
sales    = pd.read_csv(r'C:\Users\nicol\OneDrive\Desktop\Henry Data Engineer\PI1\datasets\sales.csv')
products    = pd.read_csv(r'C:\Users\nicol\OneDrive\Desktop\Henry Data Engineer\PI1\datasets\products.csv')
employees    = pd.read_csv(r'C:\Users\nicol\OneDrive\Desktop\Henry Data Engineer\PI1\datasets\employees.csv')
sales.columns = sales.columns.str.lower()
products.columns = products.columns.str.lower()
employees.columns = employees.columns.str.lower()

TASK 1 (Outliers & ML)
=============================

**Description**
- Detecta los outliers en la columna de ventas totales (TotalPriceCalculated)
- Utilizando el criterio del rango intercuartílico (IQR). Luego, crea una nueva columna llamada IsOutlier que tenga el valor 1 si el registro es un outlier y 0 en caso contrario. ¿Cuántos outliers se detectaron?

In [ ]:
# Important Features (products)
prod_cols = ['productid', 'price', 'categoryid', 'class', 'isallergic', 'vitalitydays']
products = products[prod_cols]

# Merge Sales Table
df = sales.merge(products, on='productid', how='left')

# Creating totalpricecalculated
df['totalpricecalculated'] = df['quantity'] * df['price'] * (1 - df['discount'])

# Outliers Detection
Q1    = df['totalpricecalculated'].quantile(0.25)
Q3    = df['totalpricecalculated'].quantile(0.75)
IQR   = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

df['isoutlier'] = ((df['totalpricecalculated'] < lower) | (df['totalpricecalculated'] > upper)).astype(int)
outlier_count = df['isoutlier'].sum()
print(f"Cantidad de Outliers Detectados: {outlier_count}")


# Transformaciones para ML
# - One-hot encoding en 'categoryid' y 'class' ,  Binarización de 'isallergic'
df_model = pd.get_dummies(df,columns=['categoryid','class'],drop_first=True)

df_model['isallergic_flag'] = df_model['isallergic'].map({'Yes':1,'No':0})

#Justificacion:
# One-hot para variables categoricas sin orden.
# Mapeo a 0/1 para variables binarias sencillas.
# No tocar la variable objetivo, para poder evaluar el modelo directamente en su escala original.

# Muestra de comprobación
df_model.head()


Cantidad de Outliers Detectados: 48217


,salesid,salespersonid,customerid,productid,quantity,discount,totalprice,salesdate,transactionnumber,price,...,categoryid_5.0,categoryid_6.0,categoryid_7.0,categoryid_8.0,categoryid_9.0,categoryid_10.0,categoryid_11.0,class_Low,class_Medium,isallergic_flag
0,1,6,27039,381,7,0.0,0.0,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G,44.2337,...,False,False,False,False,False,False,False,False,False,NaN
1,2,16,25011,61,7,0.0,0.0,2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8,62.5460,...,False,False,False,True,False,False,False,True,False,NaN
2,3,13,94024,23,24,0.0,0.0,2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0,79.0184,...,False,False,False,False,False,False,True,False,True,NaN
3,4,8,73966,176,19,0.2,0.0,2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE,81.3167,...,False,True,False,False,False,False,False,False,False,NaN
4,5,10,32653,310,9,0.0,0.0,2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3,79.9780,...,False,False,False,False,True,False,False,True,False,NaN


Salida de Ejemplo:

![primer respuesta](images/ta_q1.png)

TASK 2 (Sales Insight)
=============================

**Description**
- A partir de la columna SalesDate, crea una nueva columna que contenga únicamente la hora de la venta...

- Luego, identifica en qué hora del día se concentran más ventas totales (TotalPriceCalculated).

- ¿La empresa vende más durante los días de semana o en el fin de semana? Utiliza la columna SalesDate para identificar el día de la semana de cada venta, clasifica los registros como Entre semana o Fin de semana, y compara el total de ventas (TotalPriceCalculated) entre ambos grupos.

In [94]:

# Creating sale hour datapoint
df['salesdate'] = pd.to_datetime(df['salesdate'], errors='coerce')
df['sale_hour']  = df['salesdate'].dt.hour

# Sells by Hour
hourly_sales = df.groupby('sale_hour')['totalpricecalculated'].sum().reset_index().sort_values('totalpricecalculated', ascending=False)

# Hour with more Sells
hora_top = hourly_sales.iloc[0]
print(f"La hora con mayor cantidad de ventas es a las {int(hora_top.sale_hour)}, con {int(hora_top.totalpricecalculated)} ventas.")


# Creating weekday tag
df['weekday'] = df['salesdate'].dt.dayofweek + 1 
df['weekday_tag']    = np.where(df['weekday'] < 5,'Entre semana','Fin de semana')

# Sells by WeekDay Tag
daytype_sales = df.groupby('weekday_tag')['totalpricecalculated'].sum().reset_index()
daytype_sales
sem = daytype_sales.loc[daytype_sales['weekday_tag']=='Entre semana', 'totalpricecalculated'].values[0]
fin = daytype_sales.loc[daytype_sales['weekday_tag']=='Fin de semana', 'totalpricecalculated'].values[0]

if sem > fin:
    print("La empresa vende mas en los dias de semana.")
else:
    print("La empresa vende mas en los fin de semana.")


La hora con mayor cantidad de ventas es a las 16, con 179014421 ventas.
La empresa vende mas en los dias de semana.


La respuesta es:
- La hora con mayor cantidad de ventas es a las 16, con 179014421 ventas.
- La empresa vende mas en los dias de semana.

TASK 3 (Sellers Inisight)
=============================

**Description**
- Como parte del proceso de feature engineering, en el mismo df que vienes trabajando, calcula dos nuevas columnas en el dataset de ventas:
La edad del empleado al momento de su contratación y años de experiencia al momento de realizar cada venta.

- Utiliza las columnas BirthDate, HireDate (de la tabla employees) y SalesDate (de la tabla sales). Asegúrate de trabajar con fechas en formato adecuado.

In [ ]:
# Merge Employee Table
df = df.merge(
    employees[['employeeid', 'birthdate', 'hiredate']],
    left_on='salespersonid',
    right_on='employeeid',
    how='left'
)
df

# Normalizing Datetime fields
df['birthdate'] = pd.to_datetime(df['birthdate'], errors='coerce')
df['hiredate']  = pd.to_datetime(df['hiredate'], errors='coerce')


# Creating Age at Hire
df['age_at_hire'] = (df['hiredate'] - df['birthdate']).dt.days // 365

# Creating Age Experience at Sale
df['experience_at_sale'] = (df['salesdate'] - df['hiredate']).dt.days // 365


df[['salesid','salespersonid','salesdate',
          'birthdate','hiredate',
          'age_at_hire','experience_at_sale']].head()


#da un valor 0.0 para el segundo registro, se puede ajustar cambiando el divide de // a / para tomar reflejar el valor como float desde el DF, en mi caso decidi dejarlo como int.

Salida de Ejemplo:

![tercer respuesta](images/ta_q3.png)